# Embedding

In [1]:
!pip install -q langchain==0.0.219 openai==0.27.8

# !pip install -q transformers==4.29.2 sentencepiece==0.1.99 accelerate==0.19.0 bitsandbytes==0.39.0

!pip install -q python-dotenv==1.0.0

!pip install -q pandas==1.5.3
!pip install -q unstructured==0.7.12
!pip install -q wikipedia==1.4.0
!pip install -q pypdf==3.12.0
!pip install -q jq==1.4.1
!pip install -q nltk==3.8.1
!pip install -q tiktoken==0.4.0
!pip install -q transformers==4.29.2 sentencepiece==0.1.99
!pip install -q sentence-transformers==2.2.2
!pip install -q cohere==4.11.2
!pip install -q faiss-cpu==1.7.4
!pip install -q chromadb==0.3.26

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 64.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 1.5.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.1/417.1 kB 30.7 MB/s eta 0

In [4]:
from dotenv import dotenv_values
env_values = dotenv_values("./app.env")
openai_api_key = env_values['OPEN_API_KEY']

In [5]:
from langchain.llms import OpenAI
llm = OpenAI(model_name="gpt-3.5-turbo",openai_api_key=openai_api_key, temperature=0.9)


/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:189: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:769: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [6]:
from langchain.document_loaders import WikipediaLoader

query_1 = "Nikola Tesla"
query_2 = "Thomas Edison"

# ==============
docs_1 = WikipediaLoader(query=query_1, load_max_docs=1,
                         doc_content_chars_max=20_000).load()

text_document_1 = docs_1[0].page_content

# ==============
docs_2 = WikipediaLoader(query=query_2, load_max_docs=1,
                         doc_content_chars_max=20_000).load()

text_document_2 = docs_2[0].page_content

In [7]:
documents = [ text_document_1, text_document_2]
metadatas = [ {"document":query_1}, {"document":query_2} ]

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
from langchain.text_splitter import NLTKTextSplitter

text_splitter = NLTKTextSplitter(chunk_size=300, chunk_overlap=50)

text =  text_splitter.create_documents(documents, metadatas=metadatas)

In [11]:
len(text)

176

In [13]:
text[5]

Document(page_content='Attempting to develop inventions he could patent and market, Tesla conducted a range of experiments with mechanical oscillators/generators, electrical discharge tubes, and early X-ray imaging.\n\nHe also built a wirelessly controlled boat, one of the first ever exhibited.', metadata={'document': 'Nikola Tesla'})

## OpenAI

In [12]:
from langchain.embeddings import OpenAIEmbeddings

embd_llm = OpenAIEmbeddings(model="text-embedding-ada-002",openai_api_key=openai_api_key)

In [14]:
docs_text = [chunk.page_content for chunk in text ]
embeddings = embd_llm.embed_documents(docs_text)

In [15]:
# queury
token = "Hi Can I ask you about Tesla when he was born?"
print(embd_llm.embed_query(token))

[0.005466191563755274, -0.018376408144831657, 0.004074139054864645, -0.04117579013109207, -0.010583546943962574, -0.004278175067156553, -0.015480412170290947, 0.013334742747247219, -0.04317665845155716, -0.020403604954481125, 0.007187334354966879, 0.022364983335137367, 0.016572993248701096, -0.02703806757926941, 0.021864766255021095, 0.0024418507236987352, 0.023220619186758995, 0.0036594851408153772, 0.015256631188094616, 0.009938529692590237, -0.018494879826903343, -0.01650717481970787, 0.00690431660041213, 0.002132505876943469, -0.01087314635515213, 0.009918784722685814, 0.015059176832437515, -0.013058306649327278, -0.010537474416196346, -0.02445799857378006, 0.015256631188094616, -0.0004784151678904891, -0.03164533153176308, -0.003649612423032522, -0.03477827459573746, -0.023826144635677338, 0.012887179851531982, -0.0016199474921450019, 0.023115308955311775, 0.005844645667821169, 0.015756849199533463, 0.0352521650493145, -0.0049034468829631805, -0.005545173306018114, -0.013558524660

## Hugging Face

In [16]:
from langchain.embeddings import SentenceTransformerEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"

embedding_llm = SentenceTransformerEmbeddings(model_name=model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [17]:
embeddings = embedding_llm.embed_documents(docs_text)

In [19]:
print(embedding_llm.embed_query(token))

[-0.057937879115343094, 0.11507703363895416, -0.01343447808176279, 0.07966242730617523, -0.05024896562099457, -0.014208893291652203, 0.03372201323509216, 0.06129449978470802, -0.09816158562898636, -0.02242407761514187, 0.040115971118211746, -0.007319237105548382, 0.057830747216939926, -0.0391729511320591, -0.01949215680360794, 0.0014610255602747202, -0.034653931856155396, -0.02874617651104927, -0.07768731564283371, 0.022953584790229797, -0.01832769624888897, 0.024697324261069298, 0.014394130557775497, -0.03974921256303787, 0.06065879389643669, 0.026115626096725464, -0.019586658105254173, 0.05321497842669487, 0.0763850286602974, 0.005695866886526346, -0.020851289853453636, -0.013018828816711903, -0.02497638203203678, 0.018660832196474075, -0.0023525008000433445, 0.006792241241782904, 0.028159545734524727, 0.08738438785076141, 0.018522119149565697, -0.06280609965324402, -0.01926552504301071, -0.11544815450906754, -0.024547569453716278, -0.04890098422765732, 0.04978393390774727, -0.009300

##  Cohere

In [21]:
from langchain.embeddings  import CohereEmbeddings

embedding_llm = CohereEmbeddings(cohere_api_key="xQm192EgpeOpAB7I3IsECjszhEsSkqY3f8VpzSf0")

In [22]:
embeddings = embedding_llm.embed_documents(docs_text)

In [23]:
print(embedding_llm.embed_query(token))

[0.33691406, -0.29370117, 1.6708984, -1.7744141, -0.7583008, -0.29223633, -0.96484375, -1.3515625, 0.19824219, 1.0419922, 1.1445312, 0.69384766, -0.2619629, -1.2050781, -1.7919922, -0.828125, -1.0595703, -0.3515625, 1.7294922, 1.0175781, -0.13842773, -1.3320312, 0.56933594, 1.3330078, 2.3339844, 1.2148438, 0.2290039, 2.6386719, -1.0957031, 1.7373047, 1.3046875, 0.65283203, -1.4189453, -1.0283203, 0.11114502, -0.07171631, 0.5473633, -1.9560547, 1.3291016, 0.5317383, 0.19885254, -0.8334961, -1.3623047, -0.035003662, 0.39526367, 2.671875, 0.40625, 0.7495117, 0.73291016, -0.50390625, 0.082214355, -2.5019531, 0.50927734, 0.79296875, -0.052093506, 0.27075195, -1.8046875, 1.1279297, -0.29174805, 0.14660645, -1.2431641, 1.0175781, 1.2197266, 0.2685547, -0.2265625, -1.3720703, -0.6386719, -1.4775391, -0.73876953, -0.77734375, 2.046875, -1.0791016, 3.1484375, -0.026687622, 0.8613281, -1.6416016, -0.4013672, 0.24365234, -0.30249023, -2.8925781, 1.4580078, -0.56591797, -0.60498047, -1.3583984, -0.

# Indexing

## FAISS (Vector Database)

In [27]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

embedding_llm = OpenAIEmbeddings(openai_api_key=openai_api_key)
#embeddings =  embedding.embed_documents(docs_text)


In [28]:
vector = FAISS.from_documents(text, embedding_llm)

In [30]:
simialr = vector.similarity_search(token)
len(simialr)

4

In [31]:
simialr[0]

Document(page_content='There has been a resurgence in popular interest in Tesla since the 1990s.\n\n== Early years ==\n\nNikola Tesla was born an ethnic Serb in the village of Smiljan, within the Military Frontier, in the Austrian Empire (present-day Croatia), on 10 July [O.S.\n\n28 June] 1856.', metadata={'document': 'Nikola Tesla'})

## Chroma

In [32]:
from langchain.vectorstores import Chroma

# this vector database require id column with strign type

save_to_dir = "/content/wiki_chroma_db"

docs_ids = list( range( len(text) ) )
docs_ids = [ str(d) for d in docs_ids ]

vector_db = Chroma.from_documents(
                                text,
                                embedding_llm,
                                persist_directory=save_to_dir,
                                ids=docs_ids
                            )

In [33]:
sim  = vector_db.similarity_search(token)

len(sim)

4

In [34]:
sim[0]

Document(page_content='There has been a resurgence in popular interest in Tesla since the 1990s.\n\n== Early years ==\n\nNikola Tesla was born an ethnic Serb in the village of Smiljan, within the Military Frontier, in the Austrian Empire (present-day Croatia), on 10 July [O.S.\n\n28 June] 1856.', metadata={'document': 'Nikola Tesla'})

In [ ]:
# we can save and load our data through chroma